# PRS/UKB Virus Cases

In [1]:
# Imports here.
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings
warnings.filterwarnings("ignore")

#Directory in Biowulf
os.chdir('/path/to/UKB_Files')

# Loading Files

In [2]:
#Loading all the reference files
NDD_free = pd.read_csv('ALL_NDD_FREE_CONTROLS_AGE60PLUS.txt', delimiter='\t')
phenome = pd.read_csv('covariates_phenome_to_use.txt', delimiter='\t')
massive_ICD10 = pd.read_csv('massive_ICD10_ALL_table.txt', delimiter='\t', header = None)

In [3]:
#Loading all the disease files
AD = pd.read_csv("alzheimer_disease.txt", delimiter = '\t')
ALS = pd.read_csv('ALS.IDs', header = None) 
dementia = pd.read_csv('Dementia.IDs', header = None)
PD = pd.read_csv('parkinson_disease.txt', delimiter='\t')
vascular = pd.read_csv('Vascular.IDs', header = None)

In [4]:
PD

,eid,131022-0.0,131023-0.0
0,4616362,7/1/69,51
1,2254544,9/1/72,30
2,3833716,7/1/81,51
3,5034654,9/1/81,51
4,3740612,3/1/86,51
...,...,...,...
3597,3310584,11/26/20,20
3598,2744340,11/27/20,40
3599,2953476,11/30/20,40
3600,5015948,11/30/20,40


In [5]:
#Getting MS IDs
#G35 is the ICD10 code for Multiple sclerosis
G35 = massive_ICD10.loc[massive_ICD10[1] == 'G35']
list_MS = G35[0]
MS = phenome[phenome['IID'].isin(list_MS)]
MS = MS.rename(columns = {'FID': 0})
MS

,0,IID,BIRTH_YEAR,TOWNSEND,AGE_OF_RECRUIT,BATCH,GENETIC_SEX,EUROPEAN
43,1000448,1000448,1945,-2.458300,63,57,1,1
112,1001135,1001135,1952,-0.056319,56,58,0,0
454,1004557,1004557,1969,-4.622410,40,82,1,1
460,1004617,1004617,1960,-3.193010,48,30,0,1
1234,1012353,1012353,1952,-2.355580,56,71,0,1
...,...,...,...,...,...,...,...,...
500152,6001530,6001530,1957,-2.580240,52,64,0,1
501259,6012608,6012608,1949,0.068578,58,14,0,1
501848,6018490,6018490,1956,-2.657330,53,16,0,1
501868,6018696,6018696,1948,-2.591400,60,7,1,1


# Adding Disease Column

In [6]:
#Pick your NDD here
NDD = AD
ndd = "AD"

In [7]:
#Add disease column to NDD free df
NDD_free[ndd] = 0

#Drop FID, Batch, and European
NDD_free = NDD_free.drop(columns = ['FID', 'BATCH', "EUROPEAN"])

#Rename ID column
NDD_free = NDD_free.rename(columns = {'IID': 'ID'})
print("Number of controls:", len(NDD_free))
#NDD_free

Number of controls: 96390


In [8]:
#Creating df of people with AD
NDD = NDD.rename(columns = {'eid': 0})
NDD_list = list(NDD[0])
has_NDD = phenome[phenome['IID'].isin(NDD_list)]

#Only select Europeans
has_NDD = has_NDD[has_NDD['EUROPEAN'] == 1]

#Drop FID, Batch, and European
has_NDD = has_NDD.drop(columns = ['FID', 'BATCH', "EUROPEAN"])

#Rename ID column
has_NDD = has_NDD.rename(columns = {'IID': 'ID'})

#Add NDD column
has_NDD[ndd] = 1

print(f"Number of individuals with {ndd}:", len(has_NDD))

Number of individuals with AD: 2342


In [9]:
#Combine NDD_free and has_NDD
df = pd.concat([NDD_free, has_NDD])
df

,ID,BIRTH_YEAR,TOWNSEND,AGE_OF_RECRUIT,GENETIC_SEX,AD
0,1000012,1949,-4.901740,61,1,0
1,1000047,1943,-2.440140,65,0,0
2,1000068,1948,-4.377210,61,1,0
3,1000085,1944,-0.774111,65,0,0
4,1000094,1946,-1.702940,61,1,0
...,...,...,...,...,...,...
501957,6019584,1939,3.642590,69,0,1
502065,6020668,1941,-4.469670,68,0,1
502072,6020733,1940,-2.586190,69,0,1
502341,6023425,1948,-3.100560,60,1,1


# Adding ICD10 Codes to dataframe

In [10]:
#Number of codes
search_terms = pd.read_csv('/path/to/FinnGen_UKB_Codes.csv')
search_terms = search_terms.drop(columns=['number of cases - UKB'])
search_terms = search_terms.dropna()
search_terms = search_terms[['phenocode', 'UKB Codes', 'phenotype']]

phenocode_list = list(search_terms['phenocode'])
ukb_code_list = list(search_terms['UKB Codes'])
predictor_meaning = list(search_terms['phenotype'])

unique_codes = []
for code in ukb_code_list:
    unique_codes.append(code.split(','))

clean = []
for i in unique_codes[0]:
    clean.append(i.strip())
print(clean)

['A080', 'A081', 'A082', 'A083', 'A084', 'A085']


In [11]:
#search_terms

In [12]:
print("Unique codes:", len(unique_codes))
print("Phenocode_list:", len(phenocode_list))
phenocode_list

flat_list = []
for xs in unique_codes:
    for x in xs:
        flat_list.append(x.strip())
        
#print(flat_list)

Unique codes: 32
Phenocode_list: 32


In [13]:
#Finding Viral codes in ICD10 list
for code in flat_list:
    viral_ICD10 = massive_ICD10[massive_ICD10[1] == code]
    viral_ICD10 = viral_ICD10.rename(columns = {0: 'ID', 1: "Code"})
    viral_ICD10 = list(viral_ICD10["ID"])
    df[code] = np.where(df['ID'].isin(viral_ICD10), 1, 0)

In [14]:
#Checking that ICD10 columns were added
df.columns

Index(['ID', 'BIRTH_YEAR', 'TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX', 'AD',
       'A080', 'A081', 'A082', 'A083',
       ...
       'I401', 'I408', 'I409', 'I41', 'L511', 'L512', 'L518', 'L519', 'E061',
       'J00'],
      dtype='object', length=185)

In [15]:
print("Unique codes:", len(unique_codes))
#print(unique_codes)
print("Phenocode_list:", len(phenocode_list))
#print(phenocode_list)
print(unique_codes[0])
print(phenocode_list[0])

Unique codes: 32
Phenocode_list: 32
['A080', ' A081', ' A082', ' A083', ' A084', ' A085']
AB1_VIRAL_OTHER_INTEST_INFECTIONS


In [16]:
#Adding the FinnGen Groupings to the dataframe
for j in range(len(unique_codes)):
    cols = []
    for i in unique_codes[j]:
        cols.append(i.strip())

    df[phenocode_list[j]] = df[cols].sum(axis=1)
    df[phenocode_list[j]].values[df[phenocode_list[j]] > 1] = 1

In [17]:
df

,ID,BIRTH_YEAR,TOWNSEND,AGE_OF_RECRUIT,GENETIC_SEX,AD,A080,A081,A082,A083,...,J10_INFLUPNEU,J10_INFLUENZA,INFLUENZA,G6_BELLPA,AB1_VARICELLA,AB1_ZOSTER,I9_MYOCARD,L12_ERYTHEMAMULTIF,E4_THYROIDITSUBAC,J10_COLD
0,1000012,1949,-4.901740,61,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1000047,1943,-2.440140,65,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1000068,1948,-4.377210,61,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1000085,1944,-0.774111,65,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1000094,1946,-1.702940,61,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501957,6019584,1939,3.642590,69,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
502065,6020668,1941,-4.469670,68,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
502072,6020733,1940,-2.586190,69,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
502341,6023425,1948,-3.100560,60,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## ADD PRS

In [18]:
activity = pd.read_csv('/path/to/activity_prs_test.profile', sep = '\s+')
diet = pd.read_csv('/path/to/diet_prs_test.profile', sep = '\s+')
diet.head()

,FID,IID,PHENO,CNT,CNT2,SCORE
0,1000012,1000012,-9,12,5,0.004250
1,1000031,1000031,-9,12,3,0.001250
2,1000047,1000047,-9,12,5,0.004333
3,1000050,1000050,-9,12,4,0.006250
4,1000068,1000068,-9,12,4,0.003917


In [19]:
df_activity = df.merge(activity, left_on='ID', right_on='IID', how='left')
df_activity = df_activity.rename(columns = {'SCORE':'ACTIVITY'})
df_activity.head()

,ID,BIRTH_YEAR,TOWNSEND,AGE_OF_RECRUIT,GENETIC_SEX,AD,A080,A081,A082,A083,...,I9_MYOCARD,L12_ERYTHEMAMULTIF,E4_THYROIDITSUBAC,J10_COLD,FID,IID,PHENO,CNT,CNT2,ACTIVITY
0,1000012,1949,-4.901740,61,1,0,0,0,0,0,...,0,0,0,0,1000012.0,1000012.0,-9.0,14.0,7.0,-0.003071
1,1000047,1943,-2.440140,65,0,0,0,0,0,0,...,0,0,0,0,1000047.0,1000047.0,-9.0,14.0,6.0,0.008000
2,1000068,1948,-4.377210,61,1,0,0,0,0,0,...,0,0,0,0,1000068.0,1000068.0,-9.0,14.0,8.0,-0.001286
3,1000085,1944,-0.774111,65,0,0,0,0,0,0,...,0,0,0,0,1000085.0,1000085.0,-9.0,14.0,8.0,0.002714
4,1000094,1946,-1.702940,61,1,0,0,0,0,0,...,0,0,0,0,1000094.0,1000094.0,-9.0,14.0,9.0,0.008929


## Normalize ACTIVITY PRS to Z-Score

In [20]:
#Calculate mean of controls
controls = df_activity[df_activity[ndd]==0]
mean_controls = controls['ACTIVITY'].mean()
print("Control mean:", mean_controls)

#Calculate std of controls
std_controls = controls['ACTIVITY'].std()
print("Control std:", std_controls)

df_activity['Z_activity'] = (df_activity['ACTIVITY'] - mean_controls)/std_controls
df_activity.head()

Control mean: 0.0031569932051292797
Control std: 0.0036790841383057622


,ID,BIRTH_YEAR,TOWNSEND,AGE_OF_RECRUIT,GENETIC_SEX,AD,A080,A081,A082,A083,...,L12_ERYTHEMAMULTIF,E4_THYROIDITSUBAC,J10_COLD,FID,IID,PHENO,CNT,CNT2,ACTIVITY,Z_activity
0,1000012,1949,-4.901740,61,1,0,0,0,0,0,...,0,0,0,1000012.0,1000012.0,-9.0,14.0,7.0,-0.003071,-1.692928
1,1000047,1943,-2.440140,65,0,0,0,0,0,0,...,0,0,0,1000047.0,1000047.0,-9.0,14.0,6.0,0.008000,1.316362
2,1000068,1948,-4.377210,61,1,0,0,0,0,0,...,0,0,0,1000068.0,1000068.0,-9.0,14.0,8.0,-0.001286,-1.207557
3,1000085,1944,-0.774111,65,0,0,0,0,0,0,...,0,0,0,1000085.0,1000085.0,-9.0,14.0,8.0,0.002714,-0.120330
4,1000094,1946,-1.702940,61,1,0,0,0,0,0,...,0,0,0,1000094.0,1000094.0,-9.0,14.0,9.0,0.008929,1.568754


In [21]:
print("Z_activity min: ", df_activity['Z_activity'].min())
print("Z_activity max: ", df_activity['Z_activity'].max())

Z_activity min:  -3.382011048776724
Z_activity max:  4.150926214452757


## Add Diet PRS

In [22]:
df_all = df_activity.merge(diet, left_on='ID', right_on='IID', how='left')
df_all = df_all.rename(columns = {'SCORE':'DIET'})
df_all.head()

,ID,BIRTH_YEAR,TOWNSEND,AGE_OF_RECRUIT,GENETIC_SEX,AD,A080,A081,A082,A083,...,CNT_x,CNT2_x,ACTIVITY,Z_activity,FID_y,IID_y,PHENO_y,CNT_y,CNT2_y,DIET
0,1000012,1949,-4.901740,61,1,0,0,0,0,0,...,14.0,7.0,-0.003071,-1.692928,1000012.0,1000012.0,-9.0,12.0,5.0,0.004250
1,1000047,1943,-2.440140,65,0,0,0,0,0,0,...,14.0,6.0,0.008000,1.316362,1000047.0,1000047.0,-9.0,12.0,5.0,0.004333
2,1000068,1948,-4.377210,61,1,0,0,0,0,0,...,14.0,8.0,-0.001286,-1.207557,1000068.0,1000068.0,-9.0,12.0,4.0,0.003917
3,1000085,1944,-0.774111,65,0,0,0,0,0,0,...,14.0,8.0,0.002714,-0.120330,1000085.0,1000085.0,-9.0,12.0,7.0,0.004167
4,1000094,1946,-1.702940,61,1,0,0,0,0,0,...,14.0,9.0,0.008929,1.568754,1000094.0,1000094.0,-9.0,12.0,8.0,0.010000


## Normalize DIET PRS to Z-Score¶

In [23]:
#Calculate mean of controls
controls = df_all[df_all[ndd]==0]
mean_controls = controls['DIET'].mean()
print("Diet Control mean:", mean_controls)

#Calculate std of controls
std_controls = controls['DIET'].std()
print("DIET Control std:", std_controls)

df_all['Z_diet'] = (df_all['DIET'] - mean_controls)/std_controls
df_all.head()

Diet Control mean: 0.003759412362352807
DIET Control std: 0.00266769716643485


,ID,BIRTH_YEAR,TOWNSEND,AGE_OF_RECRUIT,GENETIC_SEX,AD,A080,A081,A082,A083,...,CNT2_x,ACTIVITY,Z_activity,FID_y,IID_y,PHENO_y,CNT_y,CNT2_y,DIET,Z_diet
0,1000012,1949,-4.901740,61,1,0,0,0,0,0,...,7.0,-0.003071,-1.692928,1000012.0,1000012.0,-9.0,12.0,5.0,0.004250,0.183899
1,1000047,1943,-2.440140,65,0,0,0,0,0,0,...,6.0,0.008000,1.316362,1000047.0,1000047.0,-9.0,12.0,5.0,0.004333,0.215136
2,1000068,1948,-4.377210,61,1,0,0,0,0,0,...,8.0,-0.001286,-1.207557,1000068.0,1000068.0,-9.0,12.0,4.0,0.003917,0.058949
3,1000085,1944,-0.774111,65,0,0,0,0,0,0,...,8.0,0.002714,-0.120330,1000085.0,1000085.0,-9.0,12.0,7.0,0.004167,0.152663
4,1000094,1946,-1.702940,61,1,0,0,0,0,0,...,9.0,0.008929,1.568754,1000094.0,1000094.0,-9.0,12.0,8.0,0.010000,2.339316


In [24]:
print("Z_diet min: ", df_all['Z_diet'].min())
print("Z_diet max: ", df_all['Z_diet'].max())

Z_diet min:  -2.7837051580622747
Z_diet max:  4.650935568608214


In [25]:
#Fill nan values with 0
df = df_all
df = df.fillna(0)

In [26]:
# Create lists for the regression
predictor_list = phenocode_list
predictor_meaning = predictor_meaning

# Regressions

In [27]:
# Now nothing left to do is run the regressions and call it a day. 
from statsmodels.stats.multitest import fdrcorrection
results = []

# for predictor in range(1, 10):
for predictor in range(len(predictor_list)):
  predictor_name = predictor_list[predictor]
  predictor_description = predictor_meaning[predictor]
  this_formula = ndd + "~ df['" + predictor_list[predictor] + "']" + " + AGE_OF_RECRUIT + TOWNSEND + GENETIC_SEX + Z_activity + Z_diet"
  fitted = sm.formula.glm(formula=this_formula, family=sm.families.Binomial(), data=df).fit()
  beta_coef  = fitted.params.loc["df['" + predictor_name + "']"]
  beta_se  = fitted.bse.loc["df['" + predictor_name + "']"]
  p_val = fitted.pvalues.loc["df['" + predictor_name + "']"]
  odds_ratio = np.exp(fitted.params.loc["df['" + predictor_name + "']"])
  conf = fitted.conf_int().loc["df['" + predictor_name + "']"]
  m5, m95 = np.exp(conf)
  n = sum(df[predictor_name])
  df2 = df[df[predictor_name]==1]
  n_pairs = sum(df2[ndd])  

  print(predictor_name, odds_ratio, m5, m95, p_val, n_pairs, n)
  results.append((ndd, predictor_name, predictor_description, odds_ratio, m5, m95, p_val, n_pairs))

output = pd.DataFrame(results, columns=('NDD','CODE', 'DESCRIPTION','odds_ratio', 'ci_min', "ci_max", 'P_VAL', "N_pairs"))

AB1_VIRAL_OTHER_INTEST_INFECTIONS 3.0791156525678915 1.9219269748184389 4.933045493460666 2.9133554842653913e-06 19 249
AB1_VIRAL_HEMOR_FEVER_NOS 1.0000000000000415 1.000000000000037 1.000000000000046 9.378206183917833e-77 0 0
AB1_VIRAL_WARTS 1.6525761728881572 0.8127231925076946 3.3603175501501537 0.1653535970362242 8 202
AB1_VIRAL_SKIN_MUCOUS_MEMBRANE 1.515322061783415 0.9172351970004811 2.5033938497307098 0.10465819341063899 16 430
AB1_ANOGENITAL_HERPES_SIMPLEX 1.0000000000000415 1.000000000000037 1.000000000000046 9.378206183917833e-77 0 0
J10_VIRALPNEUMO 1.3805737870442785 0.33569527402934407 5.677720626198549 0.6548719064705172 2 57
H7_HERPESKERATITIS 5.675028533822321e-09 0.0 inf 0.998778535621908 0 15
AB1_VIRAL_NOS 1.9104520341878812 1.1713393419644536 3.1159433002664176 0.009498010400831558 17 378
AB1_VIRAL_MENINGITIS 6.405931732391405e-09 0.0 inf 0.9988655316876203 0 13
AB1_VIRAL_INOTHER 2.5188623893591395 1.0173744208393118 6.236315368823427 0.04580306738980105 5 85
AB1_VIRA

In [28]:
#Only looking at codes that have at least 3 pairings
output = output[output['N_pairs'] > 2]
output

,NDD,CODE,DESCRIPTION,odds_ratio,ci_min,ci_max,P_VAL,N_pairs
0,AD,AB1_VIRAL_OTHER_INTEST_INFECTIONS,Viral and other specified intestinal infections,3.079116e+00,1.921927,4.933045,2.913355e-06,19
2,AD,AB1_VIRAL_WARTS,Viral warts,1.652576e+00,0.812723,3.360318,1.653536e-01,8
3,AD,AB1_VIRAL_SKIN_MUCOUS_MEMBRANE,Viral infections characterized by skin and muc...,1.515322e+00,0.917235,2.503394,1.046582e-01,16
7,AD,AB1_VIRAL_NOS,"Other viral diseases, not elsewhere classified",1.910452e+00,1.171339,3.115943,9.498010e-03,17
9,AD,AB1_VIRAL_INOTHER,Viral agents as the cause of diseases classifi...,2.518862e+00,1.017374,6.236315,4.580307e-02,5
10,AD,AB1_VIRAL_HEPATITIS,Viral hepatitis,4.210039e+00,2.116269,8.375318,4.201022e-05,9
11,AD,AB1_VIRAL_ENCEPHALITIS_NOS,"Viral encephalitis, not elsewhere classified/u...",2.237870e+01,5.531790,90.532391,1.308065e-05,3
12,AD,AB1_VIRAL_CNS,Viral infections of the central nervous system,5.249422e+00,1.570244,17.549139,7.086417e-03,3
13,AD,AB1_OTHER_VIRAL,Other viral diseases,2.028465e+00,1.291027,3.187131,2.155100e-03,20
15,AD,AB1_BACT_BIR_OTHER_INF_AGENTS,"Bacterial, viral and other infectious agents",2.696295e+00,2.306060,3.152566,1.689858e-35,183


In [29]:
#Adding FDR Correction

#Sort P-values
output = output.sort_values(by = "P_VAL")

#Drop Nan-values
output = output.dropna()

#FDR Correction
rejected, p_corr = fdrcorrection(output['P_VAL'], is_sorted=True)
output['P_CORR'] = p_corr
output['REJECTED'] = rejected

In [30]:
output

,NDD,CODE,DESCRIPTION,odds_ratio,ci_min,ci_max,P_VAL,N_pairs,P_CORR,REJECTED
22,AD,J10_INFLUPNEU,Influenza and pneumonia,2.599279e+00,2.256641,2.993941,4.842740e-40,231,9.685479e-39,True
15,AD,AB1_BACT_BIR_OTHER_INF_AGENTS,"Bacterial, viral and other infectious agents",2.696295e+00,2.306060,3.152566,1.689858e-35,183,1.689858e-34,True
19,AD,MENINGITIS,Meningitis,6.061492e+01,17.820631,206.175004,4.980381e-11,6,3.320254e-10,True
0,AD,AB1_VIRAL_OTHER_INTEST_INFECTIONS,Viral and other specified intestinal infections,3.079116e+00,1.921927,4.933045,2.913355e-06,19,1.456678e-05,True
11,AD,AB1_VIRAL_ENCEPHALITIS_NOS,"Viral encephalitis, not elsewhere classified/u...",2.237870e+01,5.531790,90.532391,1.308065e-05,3,5.232259e-05,True
10,AD,AB1_VIRAL_HEPATITIS,Viral hepatitis,4.210039e+00,2.116269,8.375318,4.201022e-05,9,1.400341e-04,True
13,AD,AB1_OTHER_VIRAL,Other viral diseases,2.028465e+00,1.291027,3.187131,2.155100e-03,20,6.007365e-03,True
29,AD,L12_ERYTHEMAMULTIF,Erythema multiforme,6.767382e+00,1.968789,23.261743,2.402946e-03,3,6.007365e-03,True
12,AD,AB1_VIRAL_CNS,Viral infections of the central nervous system,5.249422e+00,1.570244,17.549139,7.086417e-03,3,1.574759e-02,True
7,AD,AB1_VIRAL_NOS,"Other viral diseases, not elsewhere classified",1.910452e+00,1.171339,3.115943,9.498010e-03,17,1.899602e-02,True


In [31]:
#Save output
output.to_csv('/path/to/regression_results/' + ndd + "_PRS_virus_UKB_ALL.csv", index=False)

In [32]:
#Check results
df2 = pd.read_csv('/path/to/regression_results/' + ndd + "_PRS_virus_UKB_ALL.csv")
df2.head(25)

,NDD,CODE,DESCRIPTION,odds_ratio,ci_min,ci_max,P_VAL,N_pairs,P_CORR,REJECTED
0,AD,J10_INFLUPNEU,Influenza and pneumonia,2.599279e+00,2.256641,2.993941,4.842740e-40,231,9.685479e-39,True
1,AD,AB1_BACT_BIR_OTHER_INF_AGENTS,"Bacterial, viral and other infectious agents",2.696295e+00,2.306060,3.152566,1.689858e-35,183,1.689858e-34,True
2,AD,MENINGITIS,Meningitis,6.061492e+01,17.820631,206.175004,4.980381e-11,6,3.320254e-10,True
3,AD,AB1_VIRAL_OTHER_INTEST_INFECTIONS,Viral and other specified intestinal infections,3.079116e+00,1.921927,4.933045,2.913355e-06,19,1.456678e-05,True
4,AD,AB1_VIRAL_ENCEPHALITIS_NOS,"Viral encephalitis, not elsewhere classified/u...",2.237870e+01,5.531790,90.532391,1.308065e-05,3,5.232259e-05,True
5,AD,AB1_VIRAL_HEPATITIS,Viral hepatitis,4.210039e+00,2.116269,8.375318,4.201022e-05,9,1.400341e-04,True
6,AD,AB1_OTHER_VIRAL,Other viral diseases,2.028465e+00,1.291027,3.187131,2.155100e-03,20,6.007365e-03,True
7,AD,L12_ERYTHEMAMULTIF,Erythema multiforme,6.767382e+00,1.968789,23.261743,2.402946e-03,3,6.007365e-03,True
8,AD,AB1_VIRAL_CNS,Viral infections of the central nervous system,5.249422e+00,1.570244,17.549139,7.086417e-03,3,1.574759e-02,True
9,AD,AB1_VIRAL_NOS,"Other viral diseases, not elsewhere classified",1.910452e+00,1.171339,3.115943,9.498010e-03,17,1.899602e-02,True
